# CME 193 - Homework 1

First, save a copy of this to your own Google Drive or Github: `File` -> `Save a Copy ...` or if you are running locally `Download` -> `Download .ipynb`. 

In this assignment you will be learning about and implementing [Markov Chains](https://en.wikipedia.org/wiki/Markov_chain). If you have never heard of them before, do not worry, this notebook will explain everything needed for the assignment. 

## Markov Chains

The best way to introduce what a Markov chain is with a simple example. Suppose you have a simple counter that you can set to any number between 0 and 9 (including 0 and 9). Let us assume (although it doesn't really matter) that we start the counter at the value 1. At every step, you roll a standard 6-sided die and depending on whether the number rolled is even or odd you either increment or decrement the counter. This counter will wrap around: if you try to increment at 9, it will wrap around to 0. Similarly, if you are at 0 and decrement, it will wrap to 9. Now you can play this game for multiple steps and record the counter value at every step. This is an example of a Markov chain!

Here is an example evolution of the Markov chain, starting with the counter at 1.
- Step 1: Roll a 4. This is even, so we increment the counter from 1 to 2. 
- Step 2: Roll a 2. This is even, so we increment the counter from 2 to 3. 
- Step 3: Roll a 5. This is odd, so we decrement the counter from 3 to 2. 
- etc.

A Markov chain consists of a set of $n$ states (in our example, it was the 10 states of the counter) and a probabilistic rule to jump from one state to another each step (in our example, it was the dice rolls). Although in our example we had a similar rule for each state, you can have a different rule for each state. For example, you may say that if the counter has an odd value, then don't roll a die just always increment the counter, but to follow the die for states where the counter is even. The rule is always in the form of $n$ probabilities (one for each state) that sum to one. Each probability indicates the chance that it jumps to that state. 

This can be of several forms. For our example, we have the counter moving up by one or decreasing by one for each time step. This means that the probability of going from one state directly to another that is more than one away is 0 (i.e. probability of going from 0 directly to 2 is 0). In other examples, you could have positive probabilities between each state pairs, as long as they sum to one. 

Lets look into this more concretely for our example. Say the counter is at 0, meaning we are at state 0. Where we go next is completely determined by our dice roll. We have a 50% chance of getting an even number (2,4,6) and a 50% chance of getting an odd number (1,3,5). This means that we have a 50% chance of moving to state 1 (incrementing after an even roll) or moving to state 9 (decrementing after an odd roll). We have a 0% probability of going to the other states (2-8). Therefore, the rule for state space 0 can be represented by the following numpy array:

In [ ]:
import numpy as np 
np.array([0., 0.5, 0., 0., 0., 0., 0., 0., 0., 0.5])

Here, the index in the array represents the state and the value represents the probability with which you will jump to that state. Similarly, this would be the rule for state 4:

In [ ]:
np.array([0., 0., 0., 0.5, 0., 0.5, 0., 0., 0., 0.])

## Transition Matrix

We can represent Markov chains efficiently using a transation matrix $T$ of size $(n,n)$, where we assume <strong>column</strong> $i$ will be the rule vector for state $i$ (assuming that the states are numbered 0 to $n-1$). 

Thus, you can interpret the element $T_{i,j}$ of the matrix as the probability that you will jump to state $i$ from state $j$. 

## Question 1

Write a function to return the transition matrix for the following Markov chain, given the number of states as an argument. 

The rule for the Markov chain is as follows:
For current state $i$ (if $i$ is not $n-1$):
1. With probability 0.6, it will go to state $i+1$
2. With probability 0.1, it will go to state $i=0$
3. With probability 0.3, it will go to state $i-1$ if $i >= 1$. If $i = 0$, it stays at 0. 
4. If $i = n-1$, we stay at $i=n-1$ with probability 1.

Intuitively, this Markov chain behaves in the following way: at every step there is a 60% chance that it climbs up by 1, a 10% chance it falls down completely to 0, and a 30% chance it falls down by 1. If it does reach the final state $n-1$, it stays there forever.

You are <strong>NOT</strong> allowed to use any for-loops for this question. Instead, use `numpy` array indexing we have discussed in class to fill in your matrix. For this problem, you can assume that $n$ is at least 4. 

Hints:
1. First, write your code in an empty cell with a hardcoded $n$ value. This way, you can quickly iterate and get the right answer first. Then, you can copy it into the function.
2. You can use [array indexing](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.indexing.html#index-arrays). 

In [ ]:
##### YOUR CODE HERE
def transition_matrix(n):
    pass

In [ ]:
##### TESTING
transition_matrix(10)

To test your function, check that the above returns
```
array([[0.4, 0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0. ],
       [0.6, 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.6, 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.3, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 0.3, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.6, 0. , 0.3, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.3, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 1. ]])
```

Another intuitive check that you can do is to ensure that your columns sum to 1, as each state needs to have probabilities summing to 1. 

## Question 2
Now we want to use the transition matrix to know what the probabilities are of being in each state after running the Markov chain for $k$ steps. For example, in the above Markov chain I want to know after 30 steps, what is the probability that I have reached the final state $n-1$. 

Luckily, and maybe suprisingly, this is fairly easily to calculate using matrix multiplication. An amazingly useful property of the transition matrix is as follows:
if you have vector $p_j$ as the vector of probabilities after step $j$, then $p_{j+1}$ is actually given by the following matrix-vector product.
$$p_{j+1} = T p_j $$
We will not prove this result here, but if you spend some time thinking about what matrix multiplication is doing here, especially interpret it as taking a linear combination of columns then it should make sense. 

Thus if someone gave you the probability distribution for the starting state as $p_0$, then the probability distribution after $k$ steps is simply given by multiplying by the transition matrix $k$ times.
$$p_k = T^K p_0$$

Implement a function which takes in the initial distribution `p0`, the number of steps `k`, and a transtion matrix `tm` as parameters and returns the distribution `pk` after $k$ steps.  

In [ ]:
##### YOUR CODE HERE
def propagate(p0, k, tm):
    pass

In [ ]:
##### TESTING
tm = transition_matrix(10)
p0 = np.zeros(10)
p0[0] = 1
pk = propagate(p0, 30, tm)
pk

To check your code, the above code should return :
```
array([0.16029774, 0.13006664, 0.10561269, 0.08547506, 0.06947427,
       0.05552321, 0.04450658, 0.03281234, 0.02049648, 0.29573499])
```

## Question 3
### Part (a)

With this implementation of `propagate`, you can now compute the probability of being in each state of the Markov chain after $k$ steps. What if we wanted to know how many steps it would take for the probability of being in the final state to be at least 0.5, or 50%? 

After each step the probability of being in the final state slowly increases: once we are in state $n-1$, we never leave. At some point, this probability will cross 50% as we are more likely to increment than decrement. What you need to calculate is how many steps does it take to cross 50%. 

Again, use $n=10$ transition matrix and start from state $0$. 

Hints: 
1. Use a while loop

In [ ]:
##### YOUR CODE HERE


### Part (b)

Convert the above code instead into a function that computes the number of steps it takes to have 50% probability in the final state given the size of the Markov chain $n$.

In [ ]:
##### YOUR CODE HERE
def num_steps(n):
    pass

### Part (c)

1. Compute and plot the number of steps required to have 50% probability for all $n$ ranging from 10 to 40. 

2. Look up the plotting function `plt.semilogy` and understand what it does, and plot another graph of the same data instead using this function.
3. What does the semilogy plot tel you? 

Hint: if your code takes a long time to run, your implementation of `num_steps` may need to be improved. If you know the distrubition after $j$ steps of the Markov chain, is there a way to use it to compute the distribution after $j+1$ steps?

In [ ]:
##### YOUR CODE HERE
import matplotlib.pyplot as plt

## Question 4
### Part (a)

Now we want to also compute some samples from the Markov chain. What does this mean? Given a starting state, the Markov chain says that we move to various different states with some probabilities. We want to use randomness to choose what this next state is. Each call to the function will return a different answer, a new sample.

For example, consider the Markov chain we have been using. It computes the next state by either adding 1 (with probability 0.6), subtracting 1 (probability 0.3), or going back to state 0 (probability 0.1) until it reaches the final state. Once it reaches the final state, it stays there forever. If we were starting from state `4` of our Markov chain (for example), then we want to return `5` with probability 0.6, `3` with probability 0.3, and `0` with probability 0.1. 

Write a function `sample` which takes in a transition matrix `tm` and an initial state `s0` and samples the next state starting from `s0`.

In [ ]:
import numpy.random as rnd 

In [ ]:
##### YOUR CODE HERE
def sample(tm, s0):
    pass

Your implementation should <strong> not </strong> hard-code the probability values from the specific Markov chain we have been working with so far. 

### Part (b)
With this implementation of `sample`, write a function `sample_k_steps` which runs a random walk in the chain for `k` steps from an initial state `s0`. The random walk works in the following way: we start from `s0` and sample a new state to move to. We then set that new state to be `s0` and sample a new state to move to from there, repeating this until we have stepped `k` times. 

As an example, in the chain we have been working with so far, we can only add 1, subtract 1, or go to 0 each time we step until we are in state $n-1$ (where we stay forever once we reach it). Thus, a potential sample of 25 steps starting at state $0$ in the $n=10$ chain is
```
array([0, 1, 2, 1, 2, 3, 0, 0, 1, 2, 3, 4, 5, 6, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9, 9, 9])
```
Note that each element of this array is equal to 0, equal to $n-1$, or is exactly 1 greater or 1 less than the previous one. Further, once we have a $n-1$ value in our array then we have that value for all future elements. 

Write a function `sample_k_steps` which takes in the transition matrix `tm`, number of steps `k`, and starting state `s0` and returns an array of `k+1` states sample from the chain. 

Hint: You can use `sample` from Question 4(a) for this

In [ ]:
##### YOUR CODE HERE
def sample_k_steps(tm, k, s0):
    pass

Use the below code to test out some of the samples, it samples 20 trajectories of the Markov chain and plots them. The plot should show that almost all or all of the trajectories would have reached the final state by the end. 

In [ ]:
##### TESTING
tm = transition_matrix(10)
k = 200
s0 = 0

for i in range(20):
    states = sample_k_steps(tm, k, s0)
    plt.plot(states)


## Question 5: Empirical Consideration
Simulate the distribution of states after 100 steps using `sample_k_steps` and compare it with the result returned by `propagate`. 

1. Create a transition matrix for n = 25.
2. Sample 1000 trajectories of 100 steps using `sample_k_steps`, and record the last state for each trajectory in a separate list.
3. Plot a histogram of final states.
4. Use the function `propagate` that you wrote to compute the theoretical distribution after 100 steps.
5. Plot the theoretical distribution in the same plot and verify that it matches with the histogram.

Hints:
1. For the histogram, you can set the bins manually so there is one bin for each state. 
2. The `propagate` function returns a probability distribution, so you need to multiply it with the number of samples for it to be comparable to the histogram of samples.

In [ ]:
##### YOUR CODE HERE

## Question 6

We will now move away from Markov chains and look towards another use of linear algebra and graphs: clustering algorithms. More specifically, in this exercise we will implement a simple spectral clustering algorithm: we will demonstrate this this algorithm works on a classic synthetic dataset: the 'nested moons' dataset. We will first generate this dataset and identify the ground truth clustering we seek to find. 

In [ ]:
import sklearn.datasets

n = 1000
X,y = sklearn.datasets.make_moons(n,noise=0.05, random_state = 0)

plt.scatter(X[y==0, 0], X[y==0, 1],label="True Cluster 1")
plt.scatter(X[y==1, 0], X[y==1, 1],label="True Cluster 2")
    
    
plt.legend()

This dataset consists of $n=1000$ data points in 2-dimensional space. While it seems obvious to our eyes that a good partition of the data exists, many off-the-shelf clustering algorithms fail to fully identify the clusters on this dataset. As an example, here is how the standard K-means clustering algorithm fares on this dataset (we will see more examples of clustering in Lecture 7 when we cover Scikit-learn).

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=2, random_state=0)
kmeans_labels = model.fit(X).labels_

plt.scatter(X[kmeans_labels==0, 0], X[kmeans_labels==0, 1],label="K-Means Cluster 1")
plt.scatter(X[kmeans_labels==1, 0], X[kmeans_labels==1, 1],label="K-Means Cluster 2")
    
    
plt.legend()

### Part (a)
$\newcommand{\norm}[1]{\left\lVert#1\right\rVert}$
In this exercise, we will implement a basic spectral clustering algorithm which will correctly identify the clusters on this dataset. To begin, we will need to form an "affinity matrix": the $(i,j)$ entry of this matrix indicates how close together the $i^{th}$ and $j^{th}$ data points are in space. 

Let $x_i$ denote the $i^{th}$ point in the data set. Form an $n \times n$ matrix $A$ where $A_{ij} = e^{-10 \norm{x_i - x_j}}$ (this constant of $10$ is chosen because it gives good results for this task: it may be tuned in practice). (Here, $\norm{x_i - x_j}$ is the vector $2$-norm of the difference between $x_i$ and $x_j$.) If you did this correctly, the minimum value in $A$ should be `1.679177168057234e-14`.

In [ ]:
##### YOUR  CODE HERE

With this, we generate a [Laplacian matrix](https://www.cs.yale.edu/homes/spielman/561/2012/lect02-12.pdf). We do this in two steps: we first construct a diagonal matrix $D$ which contains the row sums of $A$ on its diagonal, and then we create the Laplacian $L = D- A$.  

In [ ]:
D =  np.diag(A.sum(axis=0))
L = D-A

The algorithmic task behind spectral clustering algorithms is to find the eigenvector associated with *second smallest* eigenvalue of the matrix $L$. (All eigenvalues of $L$ are positive, and it is a simple exercise to show that the all-ones vector is an eigenvector of $L$ with eigenvalue $0.$) As we will later see, the entries of this vector will enable us to identify the partition in our original dataset. We will build up to this functionality in stages. 

We begin by writing a linear system solver for the matrix $L$. We will use an adaptation of [Richardson iteration](https://en.wikipedia.org/wiki/Modified_Richardson_iteration) to do this task: the pseudocode for this is given below. 

```
    input) L: an n x n Laplacian matrix, b: an n x 1 vector with entries summing to 0, T: number of iterations
    outputs) x: n x 1 vector approximately satisfying Lx =b.

    x <- all-zeros vector
    steplength = 0.03
    for i = 0,1,2,...,T-1:
        residual <- b - L*x
        x <- x + steplength*residual
```

Write a function `richardson(L,b,T=500)` which implements the algorithm given above. (Note that the `steplength` parameter of $0.03$ may be optimized slightly, but this value gives good performance for solving linear systems in the Laplacian defined earlier.)

In [ ]:
##### YOUR CODE HERE
def richardson(L,b,T=500): #in our later code, we will run for T = 500 iterations
    pass

### Part (b)

While the Richardson iteration above works well on small problems, it converges slowly on larger instances. This is especially true for our Laplacian linear system: the norm of our residual error may remain higher than $10^{-4}$ even after 1000 iterations. To address this issue, we will implement a version of [Nesterov acceleration](https://calculus.subwiki.org/wiki/Nesterov%27s_gradient_acceleration) to improve the convergence of this procedure. The pseudocode for this algorithm is given below: 


```
    input) L: an n x n Laplacian matrix, b: an n x 1 vector with entries summing to 0, tol: error tolerance
    outputs) x_T: n x 1 vector approximately satisfying Lx =b.

    x_0, y_0 <- all-zeros vector
    steplength = 0.03
    alpha = 0.9
    for i = 0, 1, ... T-1:
        x_{i+1} <- y_i + steplength*(b - L*y_i)
        y_{i+1} = x_{i+1} + alpha*(x_{i+1} - x_i)
        i = i + 1
```


Here the index $i$ denotes the index of iteration: $y_1$ is a different $n$-dimensional vector to $y_0$ (and likewise with $x$). It does not indicate It may be helpful to un-roll the for loop to see the vectors it defines: if $\eta$ is the steplength, we see

\begin{align*}
x_1 = y_0 + \eta(b- L y_0) \\
y_1 = x_1 + \alpha (x_1 - x_0) \\
x_2 = y_1 + \eta (b- L y_1) \\
y_2 = x_2 + \alpha (x_2 - x_1) \\
x_3 = y_2 + \eta (b- L y_2) \\
y_3 = x_3 + \alpha (x_3 - x_2)
\end{align*}

and so on. We see that the vector $x_1$ defined on the first line is used to define both $y_1$ on the second line and $y_2$ on the fourth line. You should think of the algorithm as constructing two sequences of vectors $x_i$ and $y_i$: each $x$ vector is a function of the previous $y$ vector, but each $y$ vector is a function of the previous **two** $x$ vectors. You may find it helpful to store two different $x$ vectors (called $x_{current}$ and $x_{old}$ for example) to implement the algorithm.

(For those interested in the justification of this procedure, solving the linear equations $Lx = b$ is equivalent to finding the minimizing $x$ for the quadratic $\frac{1}{2} x^\top L x - b^\top x$. In the pseudocode, we apply the Nesterov accelerated gradient method to this problem. The constants of `alpha` and `steplength` can again be slightly improved, but these give good performance for our Laplacian)

Write a function `agd(L,b,T=500)` to implement the above pseudocode. 

In [ ]:
#YOUR CODE HERE
def agd(L,b,T=500):
    pass

In [ ]:
#TESTING-- try using Richardson and AGD to solve the linear system Lx = v
np.random.seed(0)
v = np.random.rand(n)
v -= np.sum(v)/n


If you did this correctly, you will see that the Nesterov acceleration method converges dramatically faster than Richardson iteration. 

We will now try to quantify this effect. Modify your implementations of `richardson` and `agd` to additionally return an array containing $\norm{L x_i - b}$ for each $x_i$ encountered during the while loop of the algorithm. Compare these on a semilogy plot, and label your curves appropriately. 

In [ ]:
##### YOUR CODE HERE

### part (c)


We are finally ready to apply our linear system solver to spectral clustering. We do this via a modification of the power method from Lecture 4. We provide the pseudocode for this method below:

```
    input) L: an n x n Laplacian matrix, iters: number of iterations
    outputs) l: the second-smallest eigenvalue of L
             v: the corresponding eigenvector

    v <- random vector of length n
    v <-  v - sum(v)/n 
    #this ensures v is orthogonal to the all-ones vector (the smallest eigenvalue of L) 
    for i = 0,1,2...iters-1: 
        v <- x solving the equation L*x = v
        v <-  v - sum(v)/n 
        #this ensures v is orthogonal to the all-ones vector (the smallest eigenvalue of L) 
        v <- v / ||v||_2
        l <- v^T L v
```

Here, `||v||_2` denotes the vector $2$-norm (there is a function in numpy to compute this). Write a function `pow_method(L,iters=20)` which implements the above pseudocode and solves the linear systems using your `agd` implementation from part (b). 

In [ ]:
##### YOUR CODE HERE
def pow_method(L,iters=20):
    pass

l, v = pow_method(L)

For the matrix $L$ in the problem, the eigenvalue you obtain should be between $0.1$ and $0.2$. 

We are finally ready to implement spectral clustering. To do this, use the function `np.argsort` to identify the indices of the largest $500$ entries from $v$. These will (hopefully) correspond to the elements of one of the clusters in our dataset. Additionally, compute the indices of the smallest $500$ entries of $v$: these will give the datapoints belonging to the other cluster. 

Modify the plotting code from the K-means example from earlier to display the computed spectral clusters of our input data. Label your plot appropriately.

In [ ]:
##### YOUR CODE HERE

# Submission Instructions
You need to submit this notebook (with code filled in for each question and all plots) as a pdf to Gradescope. One way to do this quickly on Colab is to do File > Print > Save to PDF. If you are running Jupyter locally, you can publish it as a pdf in the dropdown menu. 

IMPORTANT: ensure all plots needed are included in submission. You will need to tag these on Gradescope when you submit. Plots included are:
- Q3c regular and semilogy
- Q4b
- Q5
- Q6b
- Q6c